In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from importlib import reload

In [3]:
import drivers, lib
from time import sleep
import pickle

In [4]:
from lib import data_management as dm

In [5]:
# reload(drivers.Agilent_EXA)
from drivers.Agilent_EXA import *
from drivers.Agilent_PNA_L import *
# reload(drivers.KeysightAWG)
from drivers.KeysightAWG import KeysightAWG, WaveformType
from drivers.E8257D import MXG, EXG

In [6]:
exa = Agilent_EXA_N9010A("EXA")
ro_awg = KeysightAWG("AWG3")
q_awg = KeysightAWG("AWG2")
#ro_lo = EXG("EXG")

In [13]:
q_lo = MXG("MXG")

In [7]:
ro_lo = Agilent_PNA_L("PNA-L2")
ro_lo.set_frequency = ro_lo.set_center
ro_lo.set_output_state = lambda x: x
ro_lo.set_span(0)
ro_lo.set_nop(1)
ro_lo.sweep_continuous()

In [8]:
# reload(lib.iq_mixer_calibration)
from lib.iq_mixer_calibration import *

##  Readout resonator

In [9]:
ro_resonator_frequency = 7.047e9
if_offset = 0

In [10]:
cal = IQCalibrator(ro_awg, exa, ro_lo, "TEST", -6)
res = cal.calibrate(lo_frequency=ro_resonator_frequency, if_frequency=if_offset, lo_power=0,
                    ssb_power=-30, waveform_resolution=1, iterations=3, minimize_iterlimit=20,sa_res_bandwidth=500,)
#                     initial_guess=res.get_optimization_results()[0])
print(res)

Calibration data for mixer TEST
Mixer parameters: {'mixer_id': 'TEST', 'iq_attenuation': -6}
Radiation parameters: {'lo_frequency': 7047000000.0, 'lo_power': 0, 'if_frequency': 0, 'ssb_power': -30, 'waveform_resolution': 1}
Optimization results: {'dc': -91.583229064941406, 'dc_open': [-29.993877410888672]}
Optimization parameters {'dc_offsets': array([ 0.00531541, -0.06928231]), 'dc_offsets_open': array([ 0.33584961,  0.33543945]), 'if_offsets': None, 'if_amplitudes': None, 'if_phase': None}
Optimization time: 0 h 0 m 17.55 s
Finished at: 2017-05-19 10:45:38.947612


In [11]:
dm.save_IQMX_calibration(res)

In [7]:
calibrations=dm.load_IQMX_calibration_database("TEST", -20)
res = calibrations.get(frozenset(dict(lo_power=10, ssb_power=-30, lo_frequency=7e9, 
                                      if_frequency=10e6, waveform_resolution=0.1).items()))

## Qubit

In [12]:
qubit_frequency = 7.9e9
if_frequency = 50e6

In [16]:
cal = IQCalibrator(q_awg, exa, q_lo, "TESTQ", -6)
res = cal.calibrate(lo_frequency=qubit_frequency+if_frequency, if_frequency=if_frequency, lo_power=15,
                    ssb_power=-10, waveform_resolution=.1, iterations=5, minimize_iterlimit=20,sa_res_bandwidth=500,)
#                     initial_guess=res.get_optimization_results()[0])}
print(res)

Calibration data for mixer TESTQ
Mixer parameters: {'mixer_id': 'TESTQ', 'iq_attenuation': -6}
Radiation parameters: {'lo_frequency': 7950000000.0, 'lo_power': 15, 'if_frequency': 50000000.0, 'ssb_power': -10, 'waveform_resolution': 0.1}
Optimization results: {'dc': -73.562828063964844, 'if': [-10.065738677978516, -67.21025085449219, -78.24851989746094]}
Optimization parameters {'dc_offsets': array([ 0.04401699, -0.08346383]), 'dc_offsets_open': None, 'if_offsets': array([ 0.04466019, -0.0829837 ]), 'if_amplitudes': array([ 0.89104038,  0.87332281]), 'if_phase': array([ 4.64330198])}
Optimization time: 0 h 1 m 49.79 s
Finished at: 2017-05-19 10:59:37.566579


In [17]:
dm.save_IQMX_calibration(res)

In [18]:
q_lo.visa_instr.close()